# Plot Fluorescence Data

Plot fluorescence profiles from cell tracks exported by TrackMate

We begin by importing the necessary packages:

In [ ]:
import random

import matplotlib.pyplot as plt
import pandas as pd

Load the data and sort it:

In [ ]:
file_path = 'Z:/working/Training/2024-Crick Microscopy Course/Live imaging/trial data/Prelim_analysis_2/Pos0/FUCCI_LEDpower_Jul24_2_MMStack_Pos0_spots.csv'
data = pd.read_csv(file_path, skiprows=[1, 2, 3])
data.sort_values(by=['TRACK_ID', 'FRAME'], ignore_index=True, inplace=True)

Extract the relevant columns:

In [ ]:
intensity_data = data[['FRAME', 'TRACK_ID', 'MEAN_INTENSITY_CH2', 'MEAN_INTENSITY_CH3']]

Select some tracks at random:

In [ ]:
num_samples = 1  # Number of tracks to plot
random_tracks = random.sample(list(intensity_data['TRACK_ID'].unique()), num_samples)

Plot the selected fluorescence profiles

In [ ]:
%matplotlib inline
#plt.figure(figsize=(12, 6))
for track in random_tracks:
    track_data = intensity_data[intensity_data['TRACK_ID'] == track]
    plt.scatter(track_data['FRAME'], track_data['MEAN_INTENSITY_CH2'], label=f'TRACK_ID {track} - C2', alpha=0.7)
    plt.scatter(track_data['FRAME'], track_data['MEAN_INTENSITY_CH3'], label=f'TRACK_ID {track} - C3', alpha=0.7)

plt.xlabel('Frame')
plt.ylabel('Intensity')
plt.legend()
plt.show()